In [1]:
import imp
#model = imp.load_source('model', '../modules/model.py')
svhn_images = imp.load_source('svhn_images', '../modules/svhn_images.py')

In [ ]:
tf_graph = model.getGraph()
session = model.init_interactive_session(tf_graph)

In [ ]:
model.load_saved_model('/flat2/saved_models/SVHN_3_digit_May7', session)

In [ ]:
#import matplotlib.image as mpimg
#import matplotlib.pyplot as plt
#import matplotlib.patches as patches
#from matplotlib.patches import Rectangle
#from PIL import Image
#%matplotlib inline




# Download SVHN

In [ ]:
data_kind = 'test'

svhn_images.download_and_extract_SVHN(data_kind, '/flat2/sandbox')
svhn_images.separate_SVHN_images_labels('../data/SVHN/%s' % data_kind, \
                           '/home/ubuntu/Capstone/notebooks/SVHN/original_dataset/%s' %data_kind,
                           '/flat2/sandbox')

data_kind = 'train'

svhn_images.download_and_extract_SVHN(data_kind, '/flat2/sandbox')
svhn_images.separate_SVHN_images_labels('../data/SVHN/%s' % data_kind, \
                           '/home/ubuntu/Capstone/notebooks/SVHN/original_dataset/%s' % data_kind,
                           '/flat2/sandbox')